In [ ]:
import json
import minsearch

from openai import OpenAI

In [ ]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [ ]:
docs_raw

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
index.fit(documents)

In [ ]:
q = "the course has already started, can I still enroll?"

In [ ]:
boost = {"question": 3.0,
        "section": 0.5}

results = index.search(
    query=q,
    filter_dict={"course": "data-engineering-zoomcamp"}
    boost_dict=boost,
    num_results=5
)

In [ ]:
results

In [ ]:
client = OpenAI()

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": q}]
)

In [ ]:
response.choices[0].message.content

In [ ]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [ ]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

In [ ]:
print(prompt)

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}]
)

In [ ]:
response.choices[0].message.content

In [ ]:
def search(query: str,
           boost: dict=None,
           filter_dict: dict=None,
           num_results: int=5) -> list:

    results = index.search(
        query=query,
        filter_dict=filter_dict
        boost_dict=boost,
        num_results=num_results
    )

    return results

In [ ]:
def build_prompt(query: str, search_results: list) -> str:
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT: {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [ ]:
def llm(prompt: str, model: str="gpt-4o") -> str:

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content 

In [ ]:
query = "the course already started. Can I still enroll?"
search_results = search(query)
prompt = build_prompt(query=query, search_results=search_results)
answer = llm(prompt=prompt)

In [ ]:
def rag(query: str) -> str:

    search_results = search(query)
    prompt = build_prompt(query=query, search_results=search_results)
    answer = llm(prompt=prompt)

    return answer